In [121]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import csv
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [101]:
# export to csv file
def export_cols_dtype_to_csv(filename):
    df = pd.read_csv(f"{filename}")
    filename_df = pd.DataFrame(df.dtypes).reset_index().rename(columns={"index":"column", 0:"dtype"})
    filename = filename.replace(".csv","")
    with open(f"{filename}_cols_info.csv", mode="w") as csv_writer:
        csv_writer.write(filename_df.to_csv(line_terminator="\n", index=False))

export_cols_dtype_to_csv(filename="listings.csv")

In [102]:
pd.read_csv("listing_cols_info.csv").head(3)

,column,dtype
0,id,int64
1,listing_url,object
2,scrape_id,int64


In [129]:
stops = list(stopwords.words("english"))
punct = list(punctuation)

# words = "hello, i am Isaac"
# words = word_tokenize(words)
# new_word = []
# for w in words:
#     if w not in stops and w not in punct:
#         new_word.append(w)
# new_word 

In [166]:
# reviews_df = pd.read_csv("reviews.csv", encoding="utf_8")
# comments = list(reviews_df["comments"].values)
# comments[:3]

['Lovely apartment in a great location that quickly felt like home. It is always much more pleasant to return to homey apt than an anonymous hotel room. Amy is also quick to respond and a pleasure to correspond with.',
 'As a former resident of HK, I can say that Amy\'s flat is really a great flat that is a great example of Hong Kong homes in this area.  The neighborhood is very convenient for visitors to HK and also has a lot of charm and history (steps away from the British landing spot for colonial HK).\r<br/>\r<br/>And I must mention that Amy was a great "non-present" host as she has thought of everything and made a lot of provisions for the comfort of guests that would not have been included or even thought of by others.  Not to mention that the apartment is spotless and well kept.  ',
 'I stayed in this Apt for about 12 days. Such an amazing place. Right in the heart of everything. I had no problems or issues whatsoever. Amy even went out of her way to get a few misc items before

In [182]:
# # for com in comments[-1]:
# #     print(word_tokenize(["?????????"]))
# www= "aaaaa,br/,/br/"
# # www[:-5]
# if ",br/,/br/" in www:
#     www = www[:-5]
#     print(www)

aaaaa,br/


In [192]:
# def tokenize(csv_file):
reviews_df = pd.read_csv("reviews.csv", encoding="utf_8")
comments = list(reviews_df["comments"].values)

new_comments = []
for com in comments:
    if type(com) == float:
        com = "blank"
    com = com.replace("\r<br/>","")
    try:
        new_com = word_tokenize(com)
    except TypeError:
        new_com = "blank"

    token = ",".join([word for word in new_com if (word not in stops and word not in punct)])
    if len(token) <= 1:
        token = "blank"
    new_comments.append(token)

new_comments[:3]

['Lovely,apartment,great,location,quickly,felt,like,home,It,always,much,pleasant,return,homey,apt,anonymous,hotel,room,Amy,also,quick,respond,pleasure,correspond',
 "As,former,resident,HK,I,say,Amy,'s,flat,really,great,flat,great,example,Hong,Kong,homes,area,The,neighborhood,convenient,visitors,HK,also,lot,charm,history,steps,away,British,landing,spot,colonial,HK,.And,I,must,mention,Amy,great,``,non-present,'',host,thought,everything,made,lot,provisions,comfort,guests,would,included,even,thought,others,Not,mention,apartment,spotless,well,kept",
 'I,stayed,Apt,12,days,Such,amazing,place,Right,heart,everything,I,problems,issues,whatsoever,Amy,even,went,way,get,misc,items,arrival,Will,definitely,looking,staying,apartment,Thanks,incredible,stay,Hong,Kong']

In [193]:
reviews_df["token"] = new_comments
reviews_df.head(4)

,listing_id,id,date,reviewer_id,reviewer_name,comments,token
0,69074,181973,2011-02-14,358597,Chi Yan,Lovely apartment in a great location that quic...,"Lovely,apartment,great,location,quickly,felt,l..."
1,69074,252237,2011-05-05,467243,Doreen,"As a former resident of HK, I can say that Amy...","As,former,resident,HK,I,say,Amy,'s,flat,really..."
2,69074,400484,2011-07-27,696753,Michael,I stayed in this Apt for about 12 days. Such a...,"I,stayed,Apt,12,days,Such,amazing,place,Right,..."
3,69074,411022,2011-08-01,40901,Donna,"Amy's cute, comfortable apartment is in the he...","Amy,'s,cute,comfortable,apartment,heart,Sheung..."


In [263]:
reviews_df['comments'].isna().sum()

102

In [264]:
reviews_df['comments'] = reviews_df['comments'].fillna("blank")
reviews_df['comments'].isna().sum()

0

In [271]:
rere = 'Lovely apartment in a great location that quickly felt like home. It is always much more pleasant to return to homey apt than an anonymous hotel room. Amy is also quick to respond and a pleasure to correspond with.'
sia.polarity_scores(rere)

{'neg': 0.0, 'neu': 0.644, 'pos': 0.356, 'compound': 0.9565}

In [275]:
def sentiment_analyzer(review_col):
    """
    input the review series, output a new series named 'summary_sentiment' in where 'positive / neutral / negative'
    """
    summary_sentiment = []
    sia = SentimentIntensityAnalyzer()
    for comment in review_col:
        ss = sia.polarity_scores(comment)
        compound = ss['compound']

        if compound >= 0.03:
            summary_sentiment.append('positive')
        elif (compound > -0.1) and (compound < 0.03):
            summary_sentiment.append('neutral')
        else:
            summary_sentiment.append('negative')

    return summary_sentiment

reviews_df["sentiment"] = sentiment_analyzer(reviews_df["comments"])

In [276]:
with open("reviews_add_token.csv", "w", encoding="utf-8-sig") as csv_token_writer:
    csv_token_writer.write(reviews_df.to_csv(line_terminator="\n", index=False).replace("\r<br/>",""))